# Usage

Install `nbsphinx` with pip (not available yet!) or copy the file [nbsphinx.py](../nbsphinx.py) to the directory where you want to set up Sphinx (this is typically the directory where you have your notebook files).

In the same directory, run this command (assuming you have [Sphinx](http://sphinx-doc.org/) installed already):

    sphinx-quickstart
    
Answer the questions that appear on the screen, in case of doubt, just press the `<Return>` key to take the default values.

After that, there will be a few brand-new files in the current directory.
You'll have to make a few changes to the file `conf.py`.
Have a look at the [conf.py](conf.py) of `nbsphinx` for the details.

Once your `conf.py` is in place, edit the file `index.rst` and add your notebooks (without the `.ipynb` extension) to the `toctree` directive.

To create the HTML pages, use this command:

    sphinx-build <source-dir> <build-dir>
    
For example, if your source files are in the current directory, you can run this:

    sphinx-build . _build
    
Afterwards, you can find the main HTML file in `_build/index.html`.

To create LaTeX output, use:

    sphinx-build <source-dir> <build-dir> -b latex